In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
#from keras.models import Sequential
#from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
#from keras.utils.np_utils import to_categorical

#from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
#import plotly.plotly as py
#import cufflinks
from IPython.core.interactiveshell import InteractiveShell
#import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
#cufflinks.go_offline()
#cufflinks.set_config_file(world_readable=True, theme='pearl')
# Keras
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, GlobalMaxPool1D, Dense, SpatialDropout1D, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Dataset
<hr>

In [2]:
df = pd.read_csv("archive/complaints.csv",sep=",")
df.head()

,Unnamed: 0,product,narrative
0,0,credit_card,purchase order day shipping amount receive pro...
1,1,credit_card,forwarded message date tue subject please inve...
2,2,retail_banking,forwarded message cc sent friday pdt subject f...
3,3,credit_reporting,payment history missing credit report speciali...
4,4,credit_reporting,payment history missing credit report made mis...


# Data preprocessing

In [3]:
data_n = df.drop(['Unnamed: 0'], axis= 1)

In [4]:
data_n.groupby(['product']).count()

,narrative
product,
credit_card,15566
credit_reporting,91172
debt_collection,23148
mortgages_and_loans,18990
retail_banking,13535


In [5]:
import plotly.express as px
fig = px.bar(data_n.groupby(['product']).count(),
             labels={
                     "product": "Product",
                     "value": "Count of Complaint",
                     "variable": "Variable"
                 },
                title="Count Complaint in Each Product")
fig.show()

In [6]:
data_n.head()

,product,narrative
0,credit_card,purchase order day shipping amount receive pro...
1,credit_card,forwarded message date tue subject please inve...
2,retail_banking,forwarded message cc sent friday pdt subject f...
3,credit_reporting,payment history missing credit report speciali...
4,credit_reporting,payment history missing credit report made mis...


In [7]:
def print_plot(index):
    example = data_n[data_n.index == index][['narrative', 'product']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Product:', example[1])
print_plot(10)

beginning mortgage held mb financial mb mortgage portfolio purchased bank first quarter mb financial customer began fall behind mortgage payment due unemployment divorce house went foreclosure later sheriff sale home scheduled morning submitted hardship application end documentation specific date letter dated acknowledges application discussed next step never received written verbal confirmation regarding status hardship application day scheduled sheriff sale communication way sale never provided clear determination application made payment brought account current able taking loan group friend would lost home sheriff auction day sale received express mail letter dated date scheduled sheriff sale saying hardship application approved detail trial payment plan received letter timeframe required promised would take full loan friend would made required payment delay review option held numerous call since large payment told trial plan leading loan modification place payment plan fact made br

In [8]:
print_plot(70)

hello name trying reach several time get problem fixed receipted response far wrong adresses employment information exist credit account year addition also derogatory information appear credit report belong business address employment information need removed info priority expedite reported ar ar
Product: credit_reporting


In [9]:
data_n.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162421 entries, 0 to 162420
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   product    162421 non-null  object
 1   narrative  162411 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [10]:
data_n = data_n.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = str(text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    # replace REPLACE_BY_SPACE_RE symbols by space in text. 
    # substitute the matched string in REPLACE_BY_SPACE_RE with space.
    
    text = BAD_SYMBOLS_RE.sub('', text) 
    # remove symbols which are in BAD_SYMBOLS_RE from text. 
    # substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    #text = text.replace('x', '')
    #text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    # remove stopwors from text
    return text
data_n['narrative'] = data_n['narrative'].apply(clean_text)
data_n['narrative'] = data_n['narrative'].str.replace('\d+', '')

C:\Users\1557\AppData\Local\Temp\ipykernel_3564\987452026.py:27: FutureWarning:

The default value of regex will change from True to False in a future version.



In [11]:
print_plot(30)

name last four account number writing dispute late payment reported impacted covid virus hospital long time lost job spoke advised payment deferred get credit alert reported past due violation care act late payment need removed pay account full correct credit report remove erroinious late fee told provided protection care act failed adhere offered one called received written notice term deferred plan rely rep told thought protected need make right comply care act fix credit reporting remove late fee call done account negative told going forebarace would put account credit limit illegal credit ruined udapp violation care act violation
Product: credit_reporting


In [12]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, 
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', 
                      lower=True)
tokenizer.fit_on_texts(data_n['narrative'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 45960 unique tokens.


In [13]:
X = tokenizer.texts_to_sequences(data_n['narrative'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
print('Shape of data tensor:', X.shape)

Shape of data tensor: (162421, 250)


In [14]:
Y = pd.get_dummies(data_n['product']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (162421, 5)


In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(146178, 250) (146178, 5)
(16243, 250) (16243, 5)


In [16]:
## Model Architecture
def build_model():
    model = Sequential()
    
    model.add(Embedding(MAX_NB_WORDS,  EMBEDDING_DIM, input_shape=(X_train.shape[1],)))
    model.add(SpatialDropout1D(0.2))   
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Bidirectional(LSTM(128)))
    
    model.add(Dense(5,activation='softmax'))  
   
    return model

In [17]:
model = build_model()
model.summary()

C:\Users\1557\anaconda3\lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 250, 100)       │     5,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ (None, 250, 100)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 250, 512)       │       731,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 256)            │       656,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,388,805 (24.37 MB)

 Trainable params: 6,388,805 (24.37 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
num_epochs = 5
batch_size = 500
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
## For early stopping to ensure it doesnt overfit
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)
history = model.fit(X_train, y_train, 
                    epochs=num_epochs, batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss',
                                             patience=3,
                                             min_delta=0.0001)])

NameError: name 'y_train' is not defined

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
import tensorflow as tf

# Print TensorFlow version
print(tf.__version__)